# Correct viral barcodes to reduce replication, PCR, and sequencing errors
This Python Jupyter notebook uses `UMI_tools` directional adjacency method to correct viral barcodes that are likely derived from errors in the replication or sequencing library preparation process.

## Notes about UMI_tools
* Using directional adjacency method. This has been demonstrated on simulated data to produce a more accurate result than other heuristics.
* Sequence must be input as byte. See definition here: https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal
* The corrected barcode is returned as the first barcode in the group list. See umi_tools API documentation: https://umi-tools.readthedocs.io/en/latest/API.html

Import Python modules:

In [1]:
from IPython.display import display

from dms_variants.constants import CBPALETTE

import gzip

import pandas as pd

import plotnine as p9

from umi_tools import UMIClusterer

Get `snakemake` variables [as described here](https://snakemake.readthedocs.io/en/stable/snakefiles/rules.html#jupyter-notebook-integration):

In [2]:
viral_bc_in_progeny_freq_csv = 'results/viral_progeny/scProgenyProduction_trial2_viral_bc_in_progeny_freq.csv.gz'

Import barcode frequency data

In [19]:
viral_bc_df = pd.read_csv(gzip.open(viral_bc_in_progeny_freq_csv))
display(viral_bc_df)

,source,tag,gene,barcode,mean_count
0,second_infection,syn,fluHA,AAAAAAAGTAAATCTT,28.0
1,second_infection,syn,fluHA,AAAAAAATCTTAATAA,1.0
2,second_infection,syn,fluHA,AAAAAAATCTTAATGA,50.0
3,second_infection,syn,fluHA,AAAAAACAATGACTAA,0.5
4,second_infection,syn,fluHA,AAAAAACCCAATTATT,0.5
...,...,...,...,...,...
149148,supernatant,wt,fluNA,TTTTGTTAGCGTCCTG,145.5
149149,supernatant,wt,fluNA,TTTTTTAGAAAACGTA,0.5
149150,supernatant,wt,fluNA,TTTTTTAGAAAACGTC,102.0
149151,supernatant,wt,fluNA,TTTTTTCACTGCCATT,0.5


Cluster barcodes within each sample

In [4]:
clusterer = UMIClusterer(cluster_method="directional")

lookup_list = []

for (source, tag, gene), df in (viral_bc_df
                                .groupby(['source',
                                          'tag',
                                          'gene'])):
    n_unique_bc = df['barcode'].nunique()
    print(f"There are {n_unique_bc} unique barcodes in the raw data for {source} {tag} {gene}")
    
    # Convert dataframe to dictionary. Dict is requried input type for umi_tools clustering.
    viral_bc_dict = (df[['barcode','mean_count']]
                     .set_index('barcode')
                     .to_dict(orient='dict'))
    viral_bc_dict = viral_bc_dict['mean_count']
    
    # Convert barcode strings to byte. Byte is required dtype for umi_tools clustering.
    byte_dict={}
    for key, value in viral_bc_dict.items(): 
        byte_dict[key.encode("utf-8")] = float(value)
    
    # Cluster barcodes
    bc_groups = clusterer(byte_dict, threshold=1)
    groups_df = pd.DataFrame(bc_groups)
    groups_df = groups_df.stack().str.decode('utf-8').unstack() # Convert bytes back to string
    groups_df = groups_df.rename(columns={0:'corrected_bc'})
    groups_df = groups_df.set_index('corrected_bc', drop=False)
    n_corrected_bc = groups_df['corrected_bc'].nunique()
    print(f"{n_unique_bc - n_corrected_bc} barcodes were corrected.")
    print(f"There are {n_corrected_bc} corrected barcodes for {source} {tag} {gene}\n")
    
    # Generate lookup table for this sample
    temp_lookup_df = (groups_df.melt(ignore_index=False,
                                value_name='original_bc')
                      ['original_bc']
                      .dropna()
                      .reset_index())
    temp_lookup_df['source'] = source
    temp_lookup_df['tag'] = tag
    temp_lookup_df['gene'] = gene
    lookup_list.append(temp_lookup_df)

lookup_df = pd.concat(lookup_list)
display(lookup_df.describe())

There are 23862 unique barcodes in the raw data for second_infection syn fluHA
7514 barcodes were corrected.
There are 16348 corrected barcodes for second_infection syn fluHA

There are 24756 unique barcodes in the raw data for second_infection syn fluNA
5122 barcodes were corrected.
There are 19634 corrected barcodes for second_infection syn fluNA

There are 28801 unique barcodes in the raw data for second_infection wt fluHA
8566 barcodes were corrected.
There are 20235 corrected barcodes for second_infection wt fluHA

There are 22847 unique barcodes in the raw data for second_infection wt fluNA
5690 barcodes were corrected.
There are 17157 corrected barcodes for second_infection wt fluNA

There are 13478 unique barcodes in the raw data for supernatant syn fluHA
5578 barcodes were corrected.
There are 7900 corrected barcodes for supernatant syn fluHA

There are 11179 unique barcodes in the raw data for supernatant syn fluNA
5535 barcodes were corrected.
There are 5644 corrected barcod

,corrected_bc,original_bc,source,tag,gene
count,149153,149153,149153,149153,149153
unique,72115,119543,2,2,2
top,TGTTATATTTGTATTG,GGGGTGCAGAGCTTTG,second_infection,wt,fluHA
freq,132,6,100266,75878,81220


Merge corrected barcode data with barcode frequency data. Aggregate frequency on corrected barcodes.

In [20]:
viral_bc_df = pd.merge(viral_bc_df,
                       lookup_df,
                       left_on = ['source','tag','gene','barcode'],
                       right_on=['source','tag','gene','original_bc'])
viral_bc_df = viral_bc_df.drop('barcode', axis=1)

In [22]:
pd.merge(viral_bc_df,
         viral_bc_df.groupby(['source','tag','gene','corrected_bc']).sum().reset_index(),
         on=['source','tag','gene','corrected_bc'],
         suffixes=['_original_bc','_corrected_bc'])

,source,tag,gene,mean_count_original_bc,corrected_bc,original_bc,mean_count_corrected_bc
0,second_infection,syn,fluHA,28.0,AAAAAAAGTAAATCTT,AAAAAAAGTAAATCTT,28.0
1,second_infection,syn,fluHA,1.0,AAAAAAATCTTAATGA,AAAAAAATCTTAATAA,51.0
2,second_infection,syn,fluHA,50.0,AAAAAAATCTTAATGA,AAAAAAATCTTAATGA,51.0
3,second_infection,syn,fluHA,0.5,AAAGAACAATGACTAA,AAAAAACAATGACTAA,35.0
4,second_infection,syn,fluHA,34.5,AAAGAACAATGACTAA,AAAGAACAATGACTAA,35.0
...,...,...,...,...,...,...,...
149148,supernatant,wt,fluNA,134.5,TTTTCTCACTGCCATT,TTTTCTCACTGCCATT,137.0
149149,supernatant,wt,fluNA,0.5,TTTTCTCACTGCCATT,TTTTTTCACTGCCATT,137.0
149150,supernatant,wt,fluNA,89.5,TTTTGTCATGATAGCC,TTTTGTCATGATAGCC,90.0
149151,supernatant,wt,fluNA,0.5,TTTTGTCATGATAGCC,TTTTGTCATGATAGCT,90.0
